# RQ2: Best rerforming Models 

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
base_dir = "../trained_models/BagOfTricks/1000val/full/"
def load_data(train, test, fname, valtest=None) :
    if valtest == None :
        fpath = base_dir + "{}/eval/best/{}/{}.txt".format(train, test, fname)
    else :
        fpath = base_dir + "{}/val/{}/best/{}/{}.txt".format(train, valtest, test, fname)

    return np.loadtxt(fpath).astype(np.int)

train = "autoattack"
test = "pgd"

y_original = load_data(train, test, "Y_original", 1000)
y_original_pred = load_data(train, test, "Y_original_pred", 1000)
y_adv = load_data(train, test, "Y_adv", 1000)
y_adv_pred = load_data(train, test, "Y_adv_pred", 1000)

In [6]:
def get_robust_data(train, test, valtest=None):
    y_adv = load_data(train, test, "Y_adv", valtest)
    y_adv_pred = load_data(train, test, "Y_adv_pred", valtest)
    return y_adv, y_adv_pred

train = "pgd"
test = "autoattack"
y_adv, y_adv_pred = get_robust_data(train, test, None)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(pgd,autoattack): [3 8 8 ... 5 1 7]
Y_adv_pred(pgd,autoattack): [3 8 8 ... 5 1 7]


In [7]:
attacks = ["autoattack", "autopgd", "bim", "cw", "fgsm", "pgd", "deepfool", "newtonfool", "squareattack", "pixelattack", "spatialtransformation"]

In [8]:
train = attacks.copy()

test = attacks.copy()
test.append("all")

In [19]:
def get_accuracies(attacks, valtest=None) :
    predictions = {}
    labels = {}

    for a1 in train :
        p = np.array([])
        l = np.array([])
        for a2 in attacks :
            y, y_pred = get_robust_data(a1, a2, valtest)
            p = np.concatenate((p, y_pred))
            l = np.concatenate((l, y))
        predictions[a1] = {}
        labels[a1] = {}
        predictions[a1]["all"] = p
        labels[a1]["all"] = l

    for a1 in train :
        for a2 in attacks :
            y, y_pred = get_robust_data(a1, a2, valtest)
            predictions[a1][a2] = y_pred
            labels[a1][a2] = y

    accs = {}
    for tr in train :
        accs[tr] = {}
        for ts in test :
            accs[tr][ts] = (labels[tr][ts] == predictions[tr][ts]).sum().item() / len(labels[tr][ts])


    accs = pd.DataFrame(data=accs)
    accs_transpose = pd.DataFrame(data=accs).T
    return accs_transpose, accs 

accs_transpose, accs = get_accuracies(attacks, 1000)

In [20]:
accs_transpose

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
autoattack,0.722,0.721,0.714,0.723,0.702,0.713,0.710,0.727,0.732,0.119,0.364,0.632
autopgd,0.720,0.721,0.720,0.734,0.700,0.709,0.711,0.723,0.724,0.158,0.379,0.636
bim,0.715,0.713,0.719,0.726,0.701,0.716,0.709,0.721,0.721,0.111,0.325,0.625
cw,0.700,0.701,0.712,0.732,0.687,0.709,0.719,0.727,0.736,0.100,0.406,0.630
fgsm,0.705,0.704,0.708,0.728,0.707,0.709,0.708,0.716,0.706,0.103,0.287,0.616
pgd,0.721,0.722,0.720,0.727,0.719,0.722,0.717,0.719,0.723,0.136,0.345,0.634
deepfool,0.693,0.693,0.694,0.711,0.682,0.694,0.714,0.710,0.706,0.090,0.348,0.612
newtonfool,0.685,0.685,0.677,0.712,0.669,0.684,0.702,0.714,0.706,0.138,0.328,0.609
squareattack,0.641,0.641,0.650,0.696,0.643,0.635,0.650,0.683,0.771,0.111,0.317,0.585
pixelattack,0.090,0.090,0.090,0.090,0.090,0.090,0.090,0.090,0.090,0.665,0.090,0.142


## Time Execution Measurement

In [22]:
import datetime

def get_adv_train_time_execution(train) :
    fpath = base_dir + "{}/output.log".format(train)

    
    file = open(fpath)
    lines = file.readlines()
    file.close()
    
    start_time = None
    end_time = None
    
    datefmt='%Y/%m/%d %H:%M:%S'
    
    
    for l in lines :
        token = l.split("-")
        date_str = token[0][1:-2]
        date_time = datetime.datetime.strptime(date_str, datefmt)
        if token[1][1:11] == "Namespace(" :
            start_time = date_time
        else :
            end_time = date_time
    
    if end_time > start_time :
        return end_time-start_time
    raise ValueError("Duration not found")
    
_train = "autoattack"
get_adv_train_time_execution(_train) + get_adv_train_time_execution("fgsm") 

datetime.timedelta(seconds=5989)

In [36]:
durations = []
for tr in train :
    durations.append(get_adv_train_time_execution(tr))

accs_transpose["duration"] = durations

In [37]:
accs_transpose

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all,duration
autoattack,0.722,0.721,0.714,0.723,0.702,0.713,0.710,0.727,0.732,0.119,0.364,0.632,0 days 00:49:53
autopgd,0.720,0.721,0.720,0.734,0.700,0.709,0.711,0.723,0.724,0.158,0.379,0.636,0 days 00:50:11
bim,0.715,0.713,0.719,0.726,0.701,0.716,0.709,0.721,0.721,0.111,0.325,0.625,0 days 00:50:25
cw,0.700,0.701,0.712,0.732,0.687,0.709,0.719,0.727,0.736,0.100,0.406,0.630,0 days 00:50:42
fgsm,0.705,0.704,0.708,0.728,0.707,0.709,0.708,0.716,0.706,0.103,0.287,0.616,0 days 00:49:56
pgd,0.721,0.722,0.720,0.727,0.719,0.722,0.717,0.719,0.723,0.136,0.345,0.634,0 days 00:54:37
deepfool,0.693,0.693,0.694,0.711,0.682,0.694,0.714,0.710,0.706,0.090,0.348,0.612,0 days 00:50:45
newtonfool,0.685,0.685,0.677,0.712,0.669,0.684,0.702,0.714,0.706,0.138,0.328,0.609,0 days 00:54:27
squareattack,0.641,0.641,0.650,0.696,0.643,0.635,0.650,0.683,0.771,0.111,0.317,0.585,0 days 00:55:02
pixelattack,0.090,0.090,0.090,0.090,0.090,0.090,0.090,0.090,0.090,0.665,0.090,0.142,0 days 00:54:45


In [40]:
sorted_acc = accs_transpose.sort_values(by="all", ascending=False)
idx = list(sorted_acc.index)
sorted_acc = sorted_acc.reindex(idx, axis=0)
idx.append("all")
idx.append("duration")
sorted_acc = sorted_acc.reindex(idx, axis=1)
sorted_acc

,autopgd,pgd,autoattack,cw,bim,fgsm,deepfool,newtonfool,squareattack,spatialtransformation,pixelattack,all,duration
autopgd,0.721,0.709,0.720,0.734,0.720,0.700,0.711,0.723,0.724,0.379,0.158,0.636,0 days 00:50:11
pgd,0.722,0.722,0.721,0.727,0.720,0.719,0.717,0.719,0.723,0.345,0.136,0.634,0 days 00:54:37
autoattack,0.721,0.713,0.722,0.723,0.714,0.702,0.710,0.727,0.732,0.364,0.119,0.632,0 days 00:49:53
cw,0.701,0.709,0.700,0.732,0.712,0.687,0.719,0.727,0.736,0.406,0.100,0.630,0 days 00:50:42
bim,0.713,0.716,0.715,0.726,0.719,0.701,0.709,0.721,0.721,0.325,0.111,0.625,0 days 00:50:25
fgsm,0.704,0.709,0.705,0.728,0.708,0.707,0.708,0.716,0.706,0.287,0.103,0.616,0 days 00:49:56
deepfool,0.693,0.694,0.693,0.711,0.694,0.682,0.714,0.710,0.706,0.348,0.090,0.612,0 days 00:50:45
newtonfool,0.685,0.684,0.685,0.712,0.677,0.669,0.702,0.714,0.706,0.328,0.138,0.609,0 days 00:54:27
squareattack,0.641,0.635,0.641,0.696,0.650,0.643,0.650,0.683,0.771,0.317,0.111,0.585,0 days 00:55:02
spatialtransformation,0.433,0.444,0.431,0.489,0.438,0.467,0.450,0.487,0.515,0.745,0.092,0.454,0 days 00:54:59


In [41]:
selected_sorted_acc = sorted_acc[:][["all", "duration"]]
selected_sorted_acc.rename(columns={"all":"test_on_all_adv"})

,test_on_all_adv,duration
autopgd,0.636,0 days 00:50:11
pgd,0.634,0 days 00:54:37
autoattack,0.632,0 days 00:49:53
cw,0.630,0 days 00:50:42
bim,0.625,0 days 00:50:25
fgsm,0.616,0 days 00:49:56
deepfool,0.612,0 days 00:50:45
newtonfool,0.609,0 days 00:54:27
squareattack,0.585,0 days 00:55:02
spatialtransformation,0.454,0 days 00:54:59


In [42]:
# 3 classes
_clusters = {0: ['autoattack', 'autopgd', 'bim', 'cw', 'fgsm', 'pgd', 'deepfool', 'newtonfool', 'squareattack'], 1: ['pixelattack'], 2: ['spatialtransformation']}

In [43]:
clusters = []
for member in _clusters.values() :
    cluster = {}
    cluster["member"] = member
    centroid = member[0]
    acc = sorted_acc["all"][centroid]
    for x in member :
        if sorted_acc["all"][x] > acc :
            centroid = x
            acc = sorted_acc["all"][x]
    cluster["centroid"] = centroid
    clusters.append(cluster)

print(clusters)

[{'member': ['autoattack', 'autopgd', 'bim', 'cw', 'fgsm', 'pgd', 'deepfool', 'newtonfool', 'squareattack'], 'centroid': 'autopgd'}, {'member': ['pixelattack'], 'centroid': 'pixelattack'}, {'member': ['spatialtransformation'], 'centroid': 'spatialtransformation'}]


In [44]:
upper_bound = 0
n = 0
for c in clusters :
    centroid = c["centroid"]
    for m in c["member"] :
        upper_bound += sorted_acc[m][centroid]
        n += 1
print("Upper bound: ", upper_bound/n)

Upper bound:  0.7156363636363636


In [45]:
def get_execution_time(fpath) :
    
    file = open(fpath)
    lines = file.readlines()
    file.close()
    
    start_time = None
    end_time = None
    
    datefmt='%Y/%m/%d %H:%M:%S'
    
    
    for l in lines :
        token = l.split("-")
        date_str = token[0][1:-2]
        date_time = datetime.datetime.strptime(date_str, datefmt)
        if token[1][1:11] == "Namespace(" :
            start_time = date_time
        else :
            end_time = date_time
    
    if end_time > start_time :
        return end_time-start_time
    raise ValueError("Duration not found")

In [47]:
duration = None
for i in range(len(attacks)) :
    a = attacks[i]
    fpath = base_dir + "{}/output.log".format(a)
    if i == 0 :
        duration = get_execution_time(fpath)
    else :
        duration += get_execution_time(fpath)

# fpath = "noise_predictors/resnet18_3_pixelattack_spatialtransformation_autopgd_piecewise_eps8_bs128_maxlr0.05_wd0.0001_ls0.3_BNeval/output.log"
# duration += get_execution_time(fpath)

# fpath = "ensemble_models/cifar_model/eval/all/output.log"
# duration += get_execution_time(fpath)

days_and_time = duration
days = days_and_time.days
seconds = days_and_time.seconds

hours = seconds//3600
minutes = (seconds//60)%60
print("days:", days, "\nhours:", hours, "\nminutes:", minutes)

days: 0 
hours: 9 
minutes: 35
